In [1]:
# OBJETIVO:
# Obter do sítio do CARF a relação de súmulas aprovadas e vigentes,
# seu enunciado e respectivos números de acórdãos precedentes.

In [1]:
#!pip install beautifulsoup4
import pandas as pd
import re
import requests
import warnings
from bs4 import BeautifulSoup
from importlib.metadata import version

In [2]:
version('beautifulsoup4')

'4.9.3'

In [3]:
# obtém a página "Quadro Geral de Súmulas" do menu esquerdo "Jurisprudência", "Súmulas CARF"
with warnings.catch_warnings():
    # suprime o warning gerado pelo parâmetro 'verify=False' em 'requests.get'
    warnings.simplefilter('ignore')
    url = 'https://carf.economia.gov.br/jurisprudencia/sumulas-carf/quadro-geral-de-sumulas-1'
    response = requests.get(url, verify=False)

# faz o parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# regex para número de súmula, p. ex., "Súmula CARF nº 107"
re_num = re.compile(r'(\d+)')
# regex para súmula revogada, p. ex., "(Súmula revogada pela Portaria CARF nº 3, de 09/01/2018)"
re_rev = re.compile(r'(?i)Súmula revogada')
# regex para números de acórdãos precedentes, p. ex., "101-95.433, de 22/03/2006; 101-95.938, de 24/01/2007"
re_aco = re.compile(r'(\d+\-\d+\.?\d*)')

# obtém todas as tags <p> contendo "Súmula CARF"
p_sumulas = soup.find_all('p', text=re.compile(r'Súmula CARF'))
# dicionário a ser posteriormente convertido em DataFrame
# (mais eficiente do que ir apensando a um DataFrame)
sumulas = dict()
for index, p_sumula in enumerate(p_sumulas):
    num_sumula = int(re.search(re_num, p_sumula.text).group(1))
    # próxima tag <p> contém o enunciado
    p_enunciado = p_sumula.find_next_sibling('p')
    # ignora súmula revogada
    if re_rev.search(p_enunciado.text):
        print('Súmula revogada: nº', num_sumula)
        continue
    # próxima tag <p> contém o título "Acórdãos Precedentes"
    p_precedentes = p_enunciado.find_next_sibling('p')
    # próxima tag <p> contém os acórdãos precedentes propriamente ditos
    p_acordaos = p_precedentes.find_next_sibling('p')
    # constrói o dicionário {index : [num_sumula, enunciado, "num_acordao num_acordao num_acordao ..."]}
    sumulas[index] = [num_sumula, p_enunciado.text, ' '.join(re_aco.findall(p_acordaos.text))]

print('Total de súmulas extraídas:', len(sumulas))

Súmula revogada: nº 39
Súmula revogada: nº 47
Súmula revogada: nº 98
Total de súmulas extraídas: 158


In [4]:
# converte em DataFrame e exporta
df = pd.DataFrame.from_dict(sumulas, orient='index', columns=("Sumula", "Enunciado", "Precedentes"))
df.to_csv('datasets/sumulas.csv', index=False)
df.info()
df.sample()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 160
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sumula       158 non-null    int64 
 1   Enunciado    158 non-null    object
 2   Precedentes  158 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.9+ KB


,Sumula,Enunciado,Precedentes
112,113,A responsabilidade tributária do sucessor abra...,2401-004.795 3401-003.096 9101-002.212 9101-00...
